In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
nltk.download('omw-1.4')
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
def database_gen():
    # load messages dataset
    messages = pd.read_csv('messages.csv')
    #messages.head()
    #messages.isna().sum()
    # load categories dataset
    categories = pd.read_csv('categories.csv')
    #categories.head()
    #categories.isna().sum()
    # merge datasets
    df = pd.merge(messages,categories,on='id')
    #df.head()
    # create a dataframe of the 36 individual category columns
    categories = df.categories.str.split(";", expand=True)
    #categories.head()
    # select the first row of the categories dataframe
    row = categories.loc[0]
    # use this row to extract a list of new column names for categories.
    # one way is to apply a lambda function that takes everything 
    # up to the second to last character of each string with slicing
    category_colnames = row.apply(lambda x : x[:-2])
    #print(category_colnames)
    # rename the columns of `categories`
    categories.columns = category_colnames
    #categories.head()
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].apply(lambda x : int(x[-1]))
    
    # convert column from string to numeric
    #categories[column] = 
    #categories.head()
    # drop the original categories column from `df`
    df = df.drop(columns=['categories'])

    #df.head()
    # concatenate the original dataframe with the new `categories` dataframe
    df1 = pd.concat([df,categories],axis=1 ,sort='False')
    #df1
    # check number of duplicates
    duplicates=df1.duplicated().value_counts()
    #duplicates
    # drop duplicates
    df1=df1.drop_duplicates()
    # check number of duplicates
    duplicates_check=df1.duplicated().value_counts()
    #duplicates_check
    engine = create_engine('sqlite:///Disaster_Data.db')
    df1.to_sql('Messages-Categories', engine, index=False, if_exists='replace')

In [5]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///Disaster_Data.db')
    df = pd.read_sql_table('Messages-Categories',engine)
    X = df.message 
    y = df.iloc[:,4:]
    #print(len(X))
    return X, y,df

In [8]:
load_data()

(0        Weather update - a cold front from Cuba that c...
 1                  Is the Hurricane over or is it not over
 2                          Looking for someone but no name
 3        UN reports Leogane 80-90 destroyed. Only Hospi...
 4        says: west side of Haiti, rest of the country ...
                                ...                        
 26211    The training demonstrated how to enhance micro...
 26212    A suitable candidate has been selected and OCH...
 26213    Proshika, operating in Cox's Bazar municipalit...
 26214    Some 2,000 women protesting against the conduc...
 26215    A radical shift in thinking came about as a re...
 Name: message, Length: 26216, dtype: object,
        related  request  offer  aid_related  medical_help  medical_products  \
 0            1        0      0            0             0                 0   
 1            1        0      0            1             0                 0   
 2            1        0      0            0          

In [7]:
df

NameError: name 'df' is not defined

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens=list(lemmatizer.lemmatize(tok).lower().strip() for tok in tokens)
    return clean_tokens

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
class ExtraFeatures(BaseEstimator, TransformerMixin):

    def extra_features(self, text):
        sentence_list = nltk.sent_tokenize(text)
        word_list = nltk.word_tokenize(text)
        word_count = len(word_list)
        sent_count = len(sentence_list)
        char_count = len(text)
        capital_word_count = sum(map(str.isupper,text.split())) 
        return char_count, word_count, sent_count, capital_word_count

    def fit(self, x, y=None):
        return self

    def transform(self, X):         
        X_tagged = pd.Series(X).apply(self.extra_features)
        X_df = pd.DataFrame(X_tagged)
        
        return pd.DataFrame(X_df['message'].tolist(), index=X_df.index, columns=['char_count', 'word_count', 'sent_count', 'capital_word_count'])

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import re
class Word2vec(BaseEstimator, TransformerMixin):

    def word_list(self, text):
        text = re.sub(r"[^a-zA-Z0-9]", " ", text)
        text = text.lower()
        sentence_list = nltk.sent_tokenize(text)
        word_list = [nltk.word_tokenize(x) for x in sentence_list]
        return word_list

    def fit(self, x, y=None):
        return self

    def transform(self, X):         
        X_tagged = pd.Series(X).apply(self.word_list)
        X_df = pd.DataFrame(X_tagged)
        vocab = []
        for x in X_df.message:            
            vocab.extend(x)
        #print(vocab)
        from gensim.models import word2vec, Word2Vec

        num_features = 300    
        min_word_count = 3    
        num_workers = 4       
        context = 8           
        downsampling = 1e-3   

        # Initialize and train the model
        W2Vmodel = Word2Vec(sentences=vocab, sg=1, hs=0, workers=num_workers, vector_size=num_features, min_count=min_word_count, window=context,sample=downsampling, negative=5, epochs=6)
        model = W2Vmodel
        model_voc = set(model.wv.key_to_index.keys())
        sent_vector = np.zeros(model.vector_size, dtype="float32")
        
        
        
        
        dummy = np.zeros((len(X_df),num_features))
        col_names = list('w2v'+str(x) for x in range(num_features))
        df = pd.DataFrame(dummy,columns = col_names)
        
        for k, sentence in enumerate(X_df.message):
            sent_vector = np.zeros(model.vector_size, dtype="float32")
            for sentence in sentence:
                for x in sentence:                    
                    words=[]
                    words.append(x)
        
                    nwords = 0    
                    for word in words:
                        if word in model_voc: 
                            sent_vector += model.wv[word]
                            nwords += 1.    
                        if nwords > 0:
                            sent_vector /= nwords
            #print(k, sent_vector)
            df.iloc[k] = sent_vector
        return df

In [7]:
def model_1():
    
    '''Random Forest with count_vect, tfidf only'''
    pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))])  
    
  
    
    parameters = {
        'vect__ngram_range': [(1, 2)],
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1,verbose=1)

    return cv
    

    


In [8]:
def model_2():
    '''Random Forest with all features'''
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('1_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
             ('2_pipeline', Pipeline([
                ('extra', ExtraFeatures()),
                ('min_max', MinMaxScaler())
            ])),           

            ('w2v', Word2vec())
            
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    
    
    parameters = {
        'features__1_pipeline__vect__ngram_range': ((1, 2),(1,3)),
        'clf__estimator__n_estimators': [50]
        #,'clf__estimator__min_samples_split': [2, 3, 4]
    }
    
    parameters2 = {
        'features__1_pipeline__vect__ngram_range': ((1, 1),(1, 2),(1,3)),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters2,n_jobs=-1,verbose=1)
    #cv = pipeline

    return cv

In [16]:
def model_3():
    '''MNB with all features'''
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('1_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
             ('2_pipeline', Pipeline([
                ('extra', ExtraFeatures()),
                ('min_max', MinMaxScaler())
            ])),           

            ('w2v', Word2vec())
            
        ])),

        ('clf', MultiOutputClassifier(MultinomialNB()))
    ])
    
    
    
    parameters = {
        'features__1_pipeline__vect__ngram_range': [(1, 1),(1, 2),(1,3)], #reduce after confirm
        'clf__estimator__alpha': [0.05, 0.1, 1, 5,10],
        
        
    }
    
   

    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1,verbose=1)
    #cv = pipeline

    return cv

In [10]:
def display_results(cv, y_test, y_pred,name):
    import matplotlib.pyplot as plt
    precision=[]
    recall=[]
    fscore=[]
    support=[]
    #print("\nBest Parameters:", cv.best_params_)
    for i in range(len(y_test.columns)):
        #print(str(i)+": "+ y_test.columns[i]+"\n"+classification_report(y_test.iloc[:,i], y_pred[:,i]))
        a,b,c,d=score(y_test.iloc[:,i],y_pred[:,i],average='weighted')
        precision.append(a) 
        recall.append(b)
        fscore.append(c)
        support.append(d)
    accuracy = (y_pred == y_test).mean()
    #print(len(fscore))
    #print(len(accuracy))
    #print(fscore)
    a1 = np.array(accuracy)
    a2 = np.array(fscore)
    index = y_test.columns
    df = pd.DataFrame({'Accuracy'+" "+name: a1,
                   'F1_Score'+" "+name: a2}, index=index)
    
    #ax = df.plot.bar()
    
    return df

In [ ]:
def main():
    database_gen()
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42 )

    model = model_3()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    import pickle
    filename = 'model_1_final.sav'
    pickle.dump(model, open(filename, 'wb'))

    #display_results(model, y_test, y_pred)

main()

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [ ]:
# load the model from disk
filename = 'model_1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
df_model_1 = display_results(loaded_model, y_test, y_pred,filename)

In [ ]:
filename = 'model_2_param1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
df_model_2 = display_results(loaded_model, y_test, y_pred,filename)

In [ ]:
obj = pd.concat([df_model_1,df_model_2],axis=1)

In [ ]:
obj